# Writeup

The notebook contains the final writeup for our project, including visualizations from data exploration and our final model.  Our project consisted of ... [include what IPPS and DRG are]

## Style Fix

In [ ]:
%%html
<style>
table {float:left}
</style>

## Imports

In [ ]:
%matplotlib inline
import loadAndClean
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

## Load Cleaned Data

In a script we wrote, `loadAndClean.py`, we load the IPPS data and do some preprocessing on it.  We create numerical columns from values that were originally strings, numerically encode categorical features, and merge in our geocoding data.  The geocoding data is latitude and longitude values obtained by running the provider addresses through the Google Maps API.

In [ ]:
X = loadAndClean.loadAndClean()
X.head(3)

There are three columns in the data that we were interested in predicting:  _Average Covered Charges_ (how much the provider charges), _Average Total Payments_ (how much the provider actually gets paid), and _Average Medicare Payments_ (how much of that Medicare pays).  We decided to focus mainly on the third amount when developing our models because we thought it would be the easiest to predict since Medicare uses a formula to determine this amount (with exceptions for special cases).

## Data Exploration

Before creating our models, we explored and visualized different aspects of the data.  To see if it would be reasonable to predict the Medicare payment for a procedure, we looked at the data by DRG and geography.  First, since creating visuals for all 100 DRGs would be a bit much, we looked at the most common procedures to select a subset to look at.

In [ ]:
dischargeCountByDrg = []
for drg_name,drg in X.groupby('DRG Definition'):
    dischargeCountByDrg.append((drg['Total Discharges'].sum(), drg_name))
sortedDrgByCount = sorted(dischargeCountByDrg, reverse=True)

plt.figure(figsize=(15, 5))
top10 = sortedDrgByCount[:10]
plt.ylabel('Count')
plt.bar(np.arange(len(top10)), [x[0] for x in top10])
plt.xticks(np.arange(len(top10)), [x[1] for x in top10], rotation=80);

In [ ]:
drg_codes = [470, 871, 392, 292]
cols = ['Average Covered Charges Num', 'Average Total Payments Num', 'Average Medicare Payments Num']

plt.figure(figsize=(15,5*len(drg_codes)))
i = 1
for drg_code in drg_codes:
    drg = X[X['DRG Code'] == drg_code]
    drg_name = drg['DRG Definition'].values[0]
    for j,col in enumerate(cols):
        plt.subplot(len(drg_codes), 3, i)
        drg[col].hist(bins=25)
        xmin, xmax = plt.xlim()
        plt.xlim(0, xmax)
        plt.xlabel('Amount ($)')
        plt.ylabel('Count')
        plt.legend([col])
        if j == 1:
            plt.title(drg_name)
        i += 1

In [ ]:
drg_codes = [470, 871, 392, 292]
cols = ['Average Covered Charges Num', 'Average Total Payments Num', 'Average Medicare Payments Num']

plt.figure(figsize=(15,5*len(drg_codes)))
i = 1
for drg_code in drg_codes:
    drg = X[X['DRG Code'] == drg_code]
    drg_name = drg['DRG Definition'].values[0]
    for j,col in enumerate(cols):
        plt.subplot(len(drg_codes), 3, i)
        plt.hexbin(drg['Longitude'], drg['Latitude'], C=drg[col], gridsize=40)
        plt.xlabel('Latitude')
        plt.ylabel('Longitude')
        cbar = plt.colorbar(orientation='horizontal')
        cbar.set_label(col)
        plt.axis((-130,-65,25,50))
        if j == 1:
            plt.title(drg_name)
        i += 1

## Model
The model that we had the most success with was a Random Forest. [more here?]

### Cross Validation Function

In order to test our models, we created a helper function for cross validation.  It takes all of the data from 500 randomly-selected providers to use as the test data, but also checks that each procedure is present in both the train and test data.

In [ ]:
def crossVal(clf, X, predictors, y_col='Average Medicare Payments Num', cv=3):
    random.seed(2016)
    scores = []
    for i in range(cv):
        while True:
            testIds = random.sample(X['Provider Id'].unique(),500)
            testData = X[X['Provider Id'].isin(testIds)]
            trainData = X[~X['Provider Id'].isin(testIds)]
            if len(testData['DRG Code'].unique()) == len(X['DRG Code'].unique()) and len(trainData['DRG Code'].unique()) == len(X['DRG Code'].unique()):
                break
        X_train = trainData[predictors]
        y_train = trainData[y_col]
        X_test = testData[predictors]
        y_test = testData[y_col]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        scores.append(mean_squared_error(y_test, predictions)**0.5)
        print scores[i]
    print "Average RMSE: ${:,.2f}".format(np.mean(scores))

### Grouped Baseline

This baseline simply estimates the medicare payment by guessing the average cost for the given DRG code

In [ ]:
class grouped_baseline(object):
    def __init__(self):
        self.has_fit = False

    def fit(self, X_train, y_train):
        X_train = X_train.copy()
        X_train['Cost'] = y_train
        groups = X_train.groupby(['DRG Code'])

        # Average the cost for each DRG
        self.drg_costs = {}
        for ind,data in groups:
            self.drg_costs[ind] = data['Cost'].mean()

        self.has_fit = True

    def predict(self, X_test):
        if self.has_fit:
            return X_test['DRG Code'].apply(lambda x: self.drg_costs[x])
        return None

In [ ]:
alg = grouped_baseline()
crossVal(alg, X, ['DRG Code'],cv=5)

Guessing the average for the given DRG code yields a RMSE of ~$2800. We expect all of our real models to have a lower RMSE than this once they account for geographic data. 

### Best Model

After exploring different models and doing parameter sweeps. The best model we were able to find was a random forest with 200 trees, a max-depth of 22 and a min_sample_split of 18.

In [ ]:
alg = RandomForestRegressor(n_estimators=200, max_depth = 22, min_samples_split = 18, n_jobs=-1)
crossVal(alg, X, ['Latitude','Longitude','DRG Code'],cv=5)

## Conclusion

Our model performed better than the grouped baseline but not as well as we hoped. We hypothesize that the reason our error remians high is because the wage index medicare use to calculate payment is only influenced a samll amount by geography. It's possible that this model could be improved by factoring in information from another source about the area around a given hospital. 